In [70]:
import pandas as pd
import numpy as np
from urllib import request
from bs4 import BeautifulSoup

import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By

import math

In [83]:
def find_review():
    driver = webdriver.Chrome('chromedriver')
    driver.get(f_url)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    num = soup.select('#menu > div.col-detail > div.sect-grade > div.real-rating > p.desc > span > em')[0]
    n = num.text
    n = n.replace(',','')
    n = int(n)
    n = math.ceil(n / 60)

    comment_list=[]

    # 1~10 페이지까지 댓글 정보
    for i in range(1,11):
        if (i!=1):
            btn =  driver.find_element(By.CSS_SELECTOR,f'#paging_point > li:nth-child({i}) > a')
            btn.click()
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        comment = soup.select_one('#movie_point_list_container')
        cm = comment.select('.box-comment')
        for j in range(0,len(cm)):
            comment_list.append(cm[j].text)
        btn =  driver.find_element(By.CSS_SELECTOR,f'#paging_point > li:nth-child({i}) > a')
        btn.click()
    btn =  driver.find_element(By.CSS_SELECTOR,f'#paging_point > li:nth-child({11}) > button')
    btn.click()

    # 11페이지부터 댓글정보
    for i in range(1,n):
        for j in range(2,11):
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            comment = soup.select_one('#movie_point_list_container')
            cm = comment.select('.box-comment')
            for k in range(0,len(cm)):
                comment_list.append(cm[k].text)
            try:
                btn =  driver.find_element(By.CSS_SELECTOR,f'#paging_point > li:nth-child({j+2}) > a')
                btn.click()
            except:
                break
        try:        
            btn =  driver.find_element(By.CSS_SELECTOR,f'#paging_point > li:nth-child({13}) > button')
            btn.click()
        except:
            break

    global df_dict_cgv
    df_dict_cgv = dict()        
        
    df = pd.DataFrame([i for i in comment_list])
    df.columns = ['comment']
    df_dict_cgv[name]=df

    return df_dict_cgv[name]

In [84]:
def show_movie():
    driver = webdriver.Chrome('chromedriver')
    url = 'http://www.cgv.co.kr/movies/?lt=1&ft=1'
    driver.get(url)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    movie = soup.select('#contents > div.wrap-movie-chart > div.sect-movie-chart')
    m = movie[0].select('.title')
    
    movie_list=[]
    for i in range(0,len(m)):
        movie_list.append(m[i].text)
    
    global df_movie_list_cgv
    df_movie_list_cgv = pd.DataFrame([i for i in movie_list])
    df_movie_list_cgv.columns = ['name']
    
    return df_movie_list_cgv

In [85]:
def find_url():
    num = int(input('영화의 인덱스를 입력해주세요 : '))
    
    global name
    name = df_movie_list_cgv.loc[num]['name']
    
    driver = webdriver.Chrome('chromedriver')
    url = 'http://www.cgv.co.kr/movies/?lt=1&ft=1'
    driver.get(url)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    mv = soup.select('div > a')
    
    global f_url
    f_url = 'http://www.cgv.co.kr' + mv[2+num*2]['href']
    
    return None

In [86]:
show_movie()

,name
0,아바타 리마스터링
1,정직한 후보2
2,인생은 아름다워
3,공조2-인터내셔날
4,극장판 헬로카봇-수상한 마술단의 비밀
5,극장판 짱구는 못말려-수수께끼! 꽃피는 천하떡잎학교
6,늑대사냥
7,성덕
8,극장판 5등분의 신부
9,물방울을 그리는 남자


In [87]:
find_url()

영화의 인덱스를 입력해주세요 : 17


In [88]:
find_review()

,comment
0,벽력일섬 원무&폭혈\nI can do this all day.
1,총집편 4번째 작품. 익히 알듯이 재미나게 관람했다
2,역시 귀칼!! 너무 재밌다.
3,귀멸 액션은 가슴 뛰는 무엇이 있다.
4,작화는 참 좋은데말야....
...,...
109,역시 언제봐도 재밌어요
110,잘 보고 왔어요....
111,귀멸의 칼날 역시 넘 좋은 영화입니다. 감동과 재미 모두 좋아요^^
112,극장에서 보니 더 실감나고 재미있었습니다
